- To run your init.sql:
    - `brew services start mysql`
    - `mysql -uroot < init.sql`

- Overview of `auth` code
    - microservices are all running in K8s cluster
        - K8s internal network is not accessible from or to outside world by default
    - Client wants to make a request from outside the clsuter to make use of our system, via the **API gateway**
    - Gateway service will:
        - recieve request from client, and communicates with the necessary internal services to fulfill requests received from client
        - implement the upload() method to let the gateway communicate with our internal services to make this happen
    - But if our internal services are in the a private network, how can we determine whether we can let a public request into our private network?
        - The `auth` service handles this!!
        - Client credentials live in our `auth` database.
        - If they pass their check, the request is allowed to hit our backend

- Authentication scheme
    - Client provides base 64 encoded username and password in their request
    - We compare this to what we have in our mysql db
    - If match, continue. Else block request

- JSON Web Tokens (JWTs)
    - Comprises 3 JSONs that are all base64 encoded. Each base64 encoded string is concatenated together, separated by a `.`
    - Note that this isn't encrypted, only encoded
    - The 3 parts are:
        - Header: comprises the signing algorithm and the type, which is JWT
            ```json
            {
                "alg": "HS256",
                "type": "JWT"
            }
            ```
        - Payload: contains the user's claims (i.e. what the user claims to be)
            ```json
            {,
                "name": "username",
                "admin": False
            }
            ```
        - Signature: Created by taking b64 encoded header and payload, and signing them using HS256 along with the private key stored in the server 

    - What's the idea of JWTs?
        - When the client makes a request, you check if the credentials provided are valid (i.e. check against your DB record)
        - Once it is valid, you return the JWT token, which is hashed with information provided by the client AND the secret key in your server
        - If the client tries to edit his permission levels once he is authenticated, the JWT hash no longer matches
        - So you can see that, having a secret key on your server side helps ensure that the client remains honest

- Understanding `server.run(host="0.0.0.0", port=5000)`
    - Any server requires an IP address to allow external requests to be directed to it 
        - In our case, our server is a Docker container, and our application lives within that container
        - Once the Docker container spins up, it will be assigned an IP address, which can be used to send requests to our container
            - Note that this address exists only within the Docker network
    - Obviously, even if you open the address, Flask still needs to know that it has to listen to a specific IP address 
        - This is where the `host` config comes in
        - We use 0.0.0.0 to tell flask to listen to any public IP address that it can find
        - Suppose we connect our docker container to 2 networks, Docker will assign 2 separate IP addresses to this container, and 0.0.0.0 will tell flask to listen to requests coming to both

- Manifests directory
    - After the image is built, you can do the usual `docker build` `docker pull` and `docker run` 
    - In prd however, we never really want to be specifying the pull and run ourselves
    - Instead, we generate the `./manifests` directory to handle the infrastructure code for our `auth` deployment

    - Within the manifests dir, we pull the image and deploy it to Kubernetes
    
    - All of these can be deployed by simply doing `kubectl apply -f ./`